# GEE and GLMM; interpretation of marginal parameters in logistic regression models; missing data

We'll fit models with general estimating equations (`gee`) and general linear mixed models (`lme4`).

In [ ]:
library(data.table)
library(gee)
library(lme4)
library(tools)
library(xtable)

Loading required package: Matrix


## Fluoride Data

In [ ]:
head(fluoride.data <- data.table(read.csv('fluoride.csv'), key='id'))
summary(fluoride.data)

## Fluoride Data with Missing Entries

In [ ]:
head(fluoride.miss.data <- data.table(read.csv('fluoride.miss.csv'), key='id'))
summary(fluoride.miss.data)

## Models

### General Estimating Equations (GEE)

In [ ]:
gee.age.independent <- gee(fl ~ age, id=id,
                           family=binomial,
                           data=fluoride.data)
gee.age.exchangeable <- update(gee.age.independent, corstr='exchangeable')
gee.interaction.independent <- update(gee.age.independent, formula=.~. + income + age:income)
gee.interaction.exchangeable <- update(gee.interaction.independent, corstr='exchangeable')

### General Linear Mixed Models (GLMM)

In [ ]:
glmm.age <- glmer(fl ~ age + (1|id), family=binomial, data=fluoride.data)
glmm.interaction <- update(glmm.age, formula=.~. + income + age:income)

### Missing Data and GEE

In [ ]:
gee.age.independent.miss <- update(gee.age.independent, data=fluoride.miss.data)
gee.age.exchangeable.miss <- update(gee.age.exchangeable, data=fluoride.miss.data)
gee.interaction.independent.miss <- update(gee.interaction.independent, data=fluoride.miss.data)
gee.interaction.exchangeable.miss <- update(gee.interaction.exchangeable, data=fluoride.miss.data)

### Missing Data and GLMM

In [ ]:
glmm.age.miss <- update(glmm.age, data=fluoride.miss.data)
glmm.interaction.miss <- update(glmm.interaction, data=fluoride.miss.data)

### Estimates and Standard Errors

In [ ]:
summarize.model <- function(model) {
    coefficients <- summary(model)$coefficients
    standard.error <- if (is(model, 'gee')) {
        coefficients[,'Robust S.E.']
    } else if (is(model, 'glmerMod')) {
        coefficients[,'Std. Error']
    }
    data.frame(coefficient=row.names(coefficients),
               estimate=coefficients[,'Estimate'],
               standard.error=standard.error,
               row.names=NULL)    
}

key.model <- function(model) {
    data.frame(
        correlation.structure=if (is(model, 'gee')) {
            if (is.null(getCall(model)$corstr)) {
                'GEE Independent'
            } else {
                paste('GEE', toTitleCase(getCall(model)$corstr))
            }
        } else if (is(model, 'glmerMod')) {
            'Mixed Model'
        },
        has.interaction=nrow(summary(model)$coefficients) == 4,
        is.missing=getCall(model)$data == quote(fluoride.miss.data)
    )
}

(model.summaries <- do.call(rbind, lapply(list(
    gee.age.independent, gee.age.exchangeable, glmm.age,
    gee.interaction.independent, gee.interaction.exchangeable, glmm.interaction,
    gee.age.independent.miss, gee.age.exchangeable.miss, glmm.age.miss,
    gee.interaction.independent.miss, gee.interaction.exchangeable.miss, glmm.interaction.miss),
       function(model) {
           cbind(key.model(model), summarize.model(model))
       })))

write.csv(model.summaries, file='model_summaries.csv', row.names=FALSE)